# Advanced Text Analytics & Applications

## Learning Objectives
* Understand and apply Natural Language Processing concepts
* Create effective text visualisations
* Perform sentiment analysis on real text data
* Build practical text analytics solutions

## Part 1: Natural Language Processing Concepts

Let's set up our environment with the required libraries:

In [ ]:
%pip install -r ../requirements.txt nltk spacy textblob wordcloud
!python -m spacy download en_core_web_sm

In [ ]:
import nltk
import spacy
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Download required NLTK data
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker')
nltk.download('punkt_tab')
nltk.download('words')

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

print("Environment ready!")

### 1.1 Part-of-Speech Tagging

POS tagging helps identify the role of each word in a sentence:

In [ ]:
def analyse_pos(text):
    # Tokenise and tag
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)

    # Create readable output
    pos_dict = {
        'NNP': 'Proper Noun',
        'NN': 'Noun',
        'VB': 'Verb',
        'JJ': 'Adjective',
        'RB': 'Adverb'
    }

    return [(word, pos_dict.get(tag[:2], tag)) for word, tag in tagged]

# Example text
text = "Data Science helps companies make better decisions quickly."

pos_results = analyse_pos(text)
print("Part of Speech Analysis:")
for word, pos in pos_results:
    print(f"{word}: {pos}")

### 1.2 Named Entity Recognition (NER)

NER identifies entities like names, organisations, and locations:

In [ ]:
def extract_entities(text):
    # Process with spaCy
    doc = nlp(text)

    # Extract entities
    entities = {
        'ORG': [],
        'PERSON': [],
        'DATE': [],
        'GPE': []  # Geographical/Political Entities
    }

    for ent in doc.ents:
        if ent.label_ in entities:
            entities[ent.label_].append(ent.text)

    return entities

# Example text
text = """Microsoft announced new AI features in February 2024.
Sarah Johnson from Google presented at the London conference."""

entities = extract_entities(text)
print("Named Entities Found:")
for entity_type, items in entities.items():
    if items:
        print(f"{entity_type}: {', '.join(items)}")

## Part 2: Text Visualisation 

### 2.1 Word Clouds

Word clouds provide a visual representation of word frequency:

In [ ]:
def create_wordcloud(text):
    wordcloud = WordCloud(width=800, height=400,
                         background_color='white',
                         min_font_size=10).generate(text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

# Example text - multiple customer reviews
reviews = """
Great data visualisation tool! Really helps with analysis.
The machine learning features are impressive.
Data analysis has never been easier.
Excellent visualisation capabilities and machine learning support.
Great for data science projects and analysis tasks.
"""

create_wordcloud(reviews)

### 2.2 Sentiment Distribution

Visualising sentiment across multiple texts:

In [ ]:
from textblob import TextBlob
import seaborn as sns

def analyse_sentiment(text):
    return TextBlob(text).sentiment.polarity

# Example reviews
reviews = [
    "Excellent product, very satisfied!",
    "Not what I expected, disappointed.",
    "Good features but expensive.",
    "Amazing customer service!",
    "Needs improvement."
]

# Calculate sentiments
sentiments = [analyse_sentiment(review) for review in reviews]

# Create distribution plot
plt.figure(figsize=(10, 5))
sns.histplot(sentiments, bins=20)
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment Score')
plt.ylabel('Count')
plt.show()

# Print average sentiment
print(f"Average sentiment: {sum(sentiments)/len(sentiments):.2f}")

## Part 3: Applied Exercise - Customer Feedback Analysis 

Let's build a complete analysis pipeline for customer feedback:

In [ ]:
class FeedbackAnalyser:
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')

    def analyse_feedback(self, feedback):
        # Basic metrics
        doc = self.nlp(feedback)
        word_count = len([token for token in doc if not token.is_punct])

        # Sentiment
        sentiment = TextBlob(feedback).sentiment.polarity

        # Key entities
        entities = [(ent.text, ent.label_) for ent in doc.ents]

        # Key phrases (noun chunks)
        key_phrases = [chunk.text for chunk in doc.noun_chunks]

        return {
            'word_count': word_count,
            'sentiment': sentiment,
            'entities': entities,
            'key_phrases': key_phrases
        }

# Example usage
analyser = FeedbackAnalyser()

feedback = """
The new Data Science Pro software from TechCorp has transformed our analytics capabilities.
The machine learning features are impressive, though the visualisation tools need improvement.
Sarah from customer support was very helpful with the installation process.
"""

results = analyser.analyse_feedback(feedback)

print("Feedback Analysis Results:")
print(f"Word Count: {results['word_count']}")
print(f"Sentiment Score: {results['sentiment']:.2f}")
print("\nIdentified Entities:")
for entity, label in results['entities']:
    print(f"- {entity}: {label}")
print("\nKey Phrases:")
for phrase in results['key_phrases']:
    print(f"- {phrase}")

### Your Turn!

Try analyzing some text from your organisation using the FeedbackAnalyser:

In [ ]:
# Your code here
your_feedback = """
Replace this with your own text from customer feedback,
product documentation, or internal communications
"""

your_results = analyser.analyse_feedback(your_feedback)

# Analyze and visualize the results
# ...

## Summary

In this session, we've covered:
1. Advanced NLP concepts:
   - Part-of-speech tagging
   - Named Entity Recognition
2. Text visualisation techniques:
   - Word clouds
   - Sentiment distribution
3. Practical application with customer feedback analysis

### Next Steps
- Explore more advanced NLP techniques
- Try different visualisation libraries
- Apply these techniques to your own projects

### Additional Resources
- [spaCy Documentation](https://spacy.io/)
- [TextBlob Documentation](https://textblob.readthedocs.io/)
- [Seaborn Visualisation Guide](https://seaborn.pydata.org/)